# Retrieving data via bitcoin-cli

In [ ]:
# install necessary python libraries 
%pip install python-bitcoinrpc
%pip install paramiko

In [54]:
# read credentials from json file 
import pandas as pd

credentials = pd.read_json('credentials.json')

In [55]:
# establish ssh connection to the host 
import paramiko 
host = "artemis.uni.ma"
port = 22
username = credentials['user'][0]
password = credentials['pwd'][0]

ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect(host, port, username, password)

## Step 1: Monitor newly incoming files 

In [56]:
stdin, stdout, stderr = ssh.exec_command("bitcoin-cli getblockchaininfo") # returns the header hash of the most recent block on the best block chain.

In [57]:
# parse json output
import json
blockchaininfo = json.loads(stdout.read().decode("utf-8"))

bestblockhash = blockchaininfo['bestblockhash']
headers = blockchaininfo['headers']
print(bestblockhash)
print(headers)

0000000000000000000c7d08493c6e23846c1af1719e334f7e9fcf599b78797d
682811


## Step 2: Get information about the new block

In [58]:
# get blockinfo 
command = "bitcoin-cli getblock " + bestblockhash + " 2"
stdin, stdout, stderr = ssh.exec_command(command)
block = json.loads(stdout.read().decode("utf-8"))

from datetime import datetime, timezone
ts_epoch = block['time']
block_timestamp = datetime.fromtimestamp(ts_epoch)
block_date = block_timestamp.strftime('%Y-%m-%d')
block_hash = block['hash']
block_height = block['height']

# build json object
data = {}
data['block_hash'] = block_hash
data['block_height'] = block_height
data['block_timestamp'] = str(block_timestamp)
data['block_date'] = str(block_date)
data['previousblockhash'] = previousblockhash # relevant for PRECEDES relationship
data['nextblockhash'] = nextblockhash
json_data = json.dumps(data, indent=4, sort_keys=False)

print(json_data)


{
    "block_hash": "0000000000000000000c7d08493c6e23846c1af1719e334f7e9fcf599b78797d",
    "block_height": 682811,
    "block_timestamp": "2021-05-09 21:49:36",
    "block_date": "2021-05-09",
    "previousblockhash": "0000000000000000000ceb56b83c99745755126603a5ee019aa9f39b6b02268c",
    "nextblockhash": "000000000000000000080d249379f054bb42c280c81d64ced02c5592f94939ca"
}


## Step 3: Iterate through the list of txids and retrieve information on tx

In [ ]:
# parse transactions which are contained in one block 

import time
startTime = time.time() # measure execution time

for tx in block['tx']: 
    command = "bitcoin-cli getrawtransaction " + tx['txid'] + " true"
    stdin, stdout, stderr = ssh.exec_command(command)
    rawtx = json.loads(stdout.read().decode("utf-8"))
        
    # print json object = one transaction within a block
    txdata = {}
    txdata['txid'] = tx['txid']
    txdata['block_hash'] = block_hash
    txdata['block_date'] = str(block_date)
    txdata['inDegree'] = len(rawtx['vin']) 
    txdata['outDegree'] = len(rawtx['vout'])
    tx_json_data = json.dumps(txdata, indent=4, sort_keys=False)

    print(tx_json_data)
        
executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))

{
    "txid": "7154c90a0ca94eb45e0f869e8fa47c5f23744a0bbe0d83704d5f7e3131186026",
    "block_hash": "0000000000000000000c7d08493c6e23846c1af1719e334f7e9fcf599b78797d",
    "block_date": "2021-05-09",
    "inDegree": 1,
    "outDegree": 5
}
{
    "txid": "3ceb9dfb8cd47bca578659a1c9960bbb7bd7f1e2391ffb85a2802c734a484141",
    "block_hash": "0000000000000000000c7d08493c6e23846c1af1719e334f7e9fcf599b78797d",
    "block_date": "2021-05-09",
    "inDegree": 1,
    "outDegree": 11
}
{
    "txid": "d3686ddc2d72c400560335582ae777b91bfebbb75378a8fc4c15378a958596d2",
    "block_hash": "0000000000000000000c7d08493c6e23846c1af1719e334f7e9fcf599b78797d",
    "block_date": "2021-05-09",
    "inDegree": 1,
    "outDegree": 2
}
{
    "txid": "41853bbf6d1121c2e9e34e848e4c1945ee2fcdae0ed3cc182e0617b5977f63d1",
    "block_hash": "0000000000000000000c7d08493c6e23846c1af1719e334f7e9fcf599b78797d",
    "block_date": "2021-05-09",
    "inDegree": 1,
    "outDegree": 2
}
{
    "txid": "89efabac1742b2ad82554ef1

{
    "txid": "153c08c567a120c6880ca20ef256e34827da428aa0c1a2ac2629f7903e3639ff",
    "block_hash": "0000000000000000000c7d08493c6e23846c1af1719e334f7e9fcf599b78797d",
    "block_date": "2021-05-09",
    "inDegree": 1,
    "outDegree": 2
}
{
    "txid": "7c3becd8082334157e5956390b79da8bc948a5b9f266553dc4972bf80566d0ac",
    "block_hash": "0000000000000000000c7d08493c6e23846c1af1719e334f7e9fcf599b78797d",
    "block_date": "2021-05-09",
    "inDegree": 2,
    "outDegree": 2
}
{
    "txid": "99745da1c640384acf38989650fc6374053ba787d71d9d52a539222fd6a5b4af",
    "block_hash": "0000000000000000000c7d08493c6e23846c1af1719e334f7e9fcf599b78797d",
    "block_date": "2021-05-09",
    "inDegree": 1,
    "outDegree": 1
}
{
    "txid": "184f435ac5b044b411f1530fbb3d1e647c8b721db24e69e78fcf31c8c1496071",
    "block_hash": "0000000000000000000c7d08493c6e23846c1af1719e334f7e9fcf599b78797d",
    "block_date": "2021-05-09",
    "inDegree": 1,
    "outDegree": 2
}
{
    "txid": "f10385de5044a580ed1a886f9

{
    "txid": "7bf583e86468046af154c90811ef066dc3c4b5c9208a5eb9551678625013a608",
    "block_hash": "0000000000000000000c7d08493c6e23846c1af1719e334f7e9fcf599b78797d",
    "block_date": "2021-05-09",
    "inDegree": 1,
    "outDegree": 1
}
{
    "txid": "a004bbc8b405d84e12b316c004205e18c28b742a92f13d91f2514ab2127cee70",
    "block_hash": "0000000000000000000c7d08493c6e23846c1af1719e334f7e9fcf599b78797d",
    "block_date": "2021-05-09",
    "inDegree": 1,
    "outDegree": 1
}
{
    "txid": "506501380300d72c41aaa53fa76385feda67f3702beabc22a4291465cae87ca4",
    "block_hash": "0000000000000000000c7d08493c6e23846c1af1719e334f7e9fcf599b78797d",
    "block_date": "2021-05-09",
    "inDegree": 1,
    "outDegree": 1
}
{
    "txid": "4a5537cbb3318b26d1ce38626ae635aedd43d798f00995f7ec448d37738b52b4",
    "block_hash": "0000000000000000000c7d08493c6e23846c1af1719e334f7e9fcf599b78797d",
    "block_date": "2021-05-09",
    "inDegree": 1,
    "outDegree": 1
}
{
    "txid": "e203d7d2e1f6239cab11fea1c

{
    "txid": "41dab3a13c73307166a1d2a40c9085e4f7aaefc5af9804461be79d595326eb69",
    "block_hash": "0000000000000000000c7d08493c6e23846c1af1719e334f7e9fcf599b78797d",
    "block_date": "2021-05-09",
    "inDegree": 5,
    "outDegree": 2
}
{
    "txid": "af17eecfcff0137fc37ee1215a074f93bd7e671e9f7ad09b9da97a68a20b27cd",
    "block_hash": "0000000000000000000c7d08493c6e23846c1af1719e334f7e9fcf599b78797d",
    "block_date": "2021-05-09",
    "inDegree": 8,
    "outDegree": 1
}
{
    "txid": "2bae084686085ee6dec2a6b9767e698b1a674737d7f00b709c2bfb8615391905",
    "block_hash": "0000000000000000000c7d08493c6e23846c1af1719e334f7e9fcf599b78797d",
    "block_date": "2021-05-09",
    "inDegree": 1,
    "outDegree": 2
}
{
    "txid": "db0d087a0de2b52c98805dc9f14a215d53814ce9c0180c4287682a68cf25690f",
    "block_hash": "0000000000000000000c7d08493c6e23846c1af1719e334f7e9fcf599b78797d",
    "block_date": "2021-05-09",
    "inDegree": 3,
    "outDegree": 2
}
{
    "txid": "081ecdc585df997654a3abaea

}
{
    "txid": "c60c6da40618036d4f42736d1fca512febd8db21840aed4fdc429cff774a513d",
    "block_hash": "0000000000000000000c7d08493c6e23846c1af1719e334f7e9fcf599b78797d",
    "block_date": "2021-05-09",
    "inDegree": 1,
    "outDegree": 1
}
{
    "txid": "ede946ccb19afe02efa6b16f09b4e6ed9b33069aa1c8e6c9e4a66a0e69914266",
    "block_hash": "0000000000000000000c7d08493c6e23846c1af1719e334f7e9fcf599b78797d",
    "block_date": "2021-05-09",
    "inDegree": 1,
    "outDegree": 2
}
{
    "txid": "32573dd1cf28f93f9c7d51a84cd09b39c27bdb3a798156b2a40fe57fa184fe2a",
    "block_hash": "0000000000000000000c7d08493c6e23846c1af1719e334f7e9fcf599b78797d",
    "block_date": "2021-05-09",
    "inDegree": 1,
    "outDegree": 1
}
{
    "txid": "3a82ad0d93f424143a2f61d736dbb486c85ac0a45087177b78f4acc95f4e12e7",
    "block_hash": "0000000000000000000c7d08493c6e23846c1af1719e334f7e9fcf599b78797d",
    "block_date": "2021-05-09",
    "inDegree": 1,
    "outDegree": 1
}
{
    "txid": "06573fec3261f1741e6958e

...

{
    "txid": "b2689ea9c0277afd0cfd2898c58c0d0d7f1c467f750f06b9ab01edb9f7a609bd",
    "block_hash": "0000000000000000000c7d08493c6e23846c1af1719e334f7e9fcf599b78797d",
    "block_date": "2021-05-09",
    "inDegree": 5,
    "outDegree": 2
}
{
    "txid": "63c87f76cc8060f1a505d11c328996c015c420b78ae1d80e42e4835b75b40fa8",
    "block_hash": "0000000000000000000c7d08493c6e23846c1af1719e334f7e9fcf599b78797d",
    "block_date": "2021-05-09",
    "inDegree": 3,
    "outDegree": 2
}
{
    "txid": "4c198ede2730d1fa5ad5706d94d43253cdb85dfc32d93ae9091e40ed8e8bf39d",
    "block_hash": "0000000000000000000c7d08493c6e23846c1af1719e334f7e9fcf599b78797d",
    "block_date": "2021-05-09",
    "inDegree": 1,
    "outDegree": 2
}
{
    "txid": "a20583dcb56a4ef910a093399690aef4572421073cb73fe1e0430b713e78667a",
    "block_hash": "0000000000000000000c7d08493c6e23846c1af1719e334f7e9fcf599b78797d",
    "block_date": "2021-05-09",
    "inDegree": 1,
    "outDegree": 2
}
{
    "txid": "c8e82c709adf155d25c1cd5e9

{
    "txid": "735ce273152d1c928c018253545955e6578cdda5d181a15256001563a23e80e3",
    "block_hash": "0000000000000000000c7d08493c6e23846c1af1719e334f7e9fcf599b78797d",
    "block_date": "2021-05-09",
    "inDegree": 1,
    "outDegree": 2
}
{
    "txid": "e8a8a95f3b6ddeb26e90648408a6bfbb93a5dd9b677436960763f1b0e15b8c3a",
    "block_hash": "0000000000000000000c7d08493c6e23846c1af1719e334f7e9fcf599b78797d",
    "block_date": "2021-05-09",
    "inDegree": 1,
    "outDegree": 2
}
{
    "txid": "412f0bc4c03eb4118bd86b6949354b2a0a0d59b89a72c41e427811454e9cd93d",
    "block_hash": "0000000000000000000c7d08493c6e23846c1af1719e334f7e9fcf599b78797d",
    "block_date": "2021-05-09",
    "inDegree": 1,
    "outDegree": 2
}
{
    "txid": "823ee21dace7d1aad1d7dc0873917f0999e7db512de8eacee1b32b3cca606d47",
    "block_hash": "0000000000000000000c7d08493c6e23846c1af1719e334f7e9fcf599b78797d",
    "block_date": "2021-05-09",
    "inDegree": 1,
    "outDegree": 2
}
{
    "txid": "e9f64e47e519e2e5a7f947f07

{
    "txid": "8abc469f4e3e53019c670b94c8e734fa124976eccaaaa906e6632ca90384c12d",
    "block_hash": "0000000000000000000c7d08493c6e23846c1af1719e334f7e9fcf599b78797d",
    "block_date": "2021-05-09",
    "inDegree": 1,
    "outDegree": 2
}
{
    "txid": "0c9f77e7ebf24069f812c8076127bc1f3c878add4f0e441a40c8bd762a609035",
    "block_hash": "0000000000000000000c7d08493c6e23846c1af1719e334f7e9fcf599b78797d",
    "block_date": "2021-05-09",
    "inDegree": 1,
    "outDegree": 2
}
{
    "txid": "036f546e3d040f10ae06928126c77d153e5d37318c3e35f9b000b4753b08ce37",
    "block_hash": "0000000000000000000c7d08493c6e23846c1af1719e334f7e9fcf599b78797d",
    "block_date": "2021-05-09",
    "inDegree": 1,
    "outDegree": 2
}
{
    "txid": "06caa15c53dc2d4a9f090e414444071295b11fe9951f054836ffe96cb4de103c",
    "block_hash": "0000000000000000000c7d08493c6e23846c1af1719e334f7e9fcf599b78797d",
    "block_date": "2021-05-09",
    "inDegree": 1,
    "outDegree": 2
}
{
    "txid": "e0b37c053aaeb702a0a78e923

{
    "txid": "506684cf0ede5cee3f1d200ee9a4fea8d5a9c13458d100743a9186855a573f86",
    "block_hash": "0000000000000000000c7d08493c6e23846c1af1719e334f7e9fcf599b78797d",
    "block_date": "2021-05-09",
    "inDegree": 1,
    "outDegree": 2
}
{
    "txid": "9e59b0929f57f3ca655adbef6310740f5230cb438b69a04e0f28fcbe0c543f88",
    "block_hash": "0000000000000000000c7d08493c6e23846c1af1719e334f7e9fcf599b78797d",
    "block_date": "2021-05-09",
    "inDegree": 1,
    "outDegree": 2
}
{
    "txid": "b33bed9978487e57f6baeecbe3685880ac2ce3849f4dca72c94c941286fabb88",
    "block_hash": "0000000000000000000c7d08493c6e23846c1af1719e334f7e9fcf599b78797d",
    "block_date": "2021-05-09",
    "inDegree": 1,
    "outDegree": 2
}
{
    "txid": "ff1b21d8bdd0be3d8b787e1b7de787bcf7282af69fb40261652f3b6ceaca378b",
    "block_hash": "0000000000000000000c7d08493c6e23846c1af1719e334f7e9fcf599b78797d",
    "block_date": "2021-05-09",
    "inDegree": 1,
    "outDegree": 2
}
{
    "txid": "3453034fe3d6d9b17826d6934

{
    "txid": "17400bb71e1fc0b35009c8a97b3e2e6c9787c5ae876d625f11a1c57a0c5adf48",
    "block_hash": "0000000000000000000c7d08493c6e23846c1af1719e334f7e9fcf599b78797d",
    "block_date": "2021-05-09",
    "inDegree": 1,
    "outDegree": 2
}
{
    "txid": "2e269dcb609e01ca00c36db698b35c91ee02214f615eb36d1b72607f825c1ec9",
    "block_hash": "0000000000000000000c7d08493c6e23846c1af1719e334f7e9fcf599b78797d",
    "block_date": "2021-05-09",
    "inDegree": 1,
    "outDegree": 2
}
{
    "txid": "3824da6462f1a21cd8a67b2fc8928033a8154d043345317fd716c1370feeb8c9",
    "block_hash": "0000000000000000000c7d08493c6e23846c1af1719e334f7e9fcf599b78797d",
    "block_date": "2021-05-09",
    "inDegree": 1,
    "outDegree": 2
}
{
    "txid": "ab25d91cf8e29a70e8d5f9198c0df4c70aa4ca90a4fef4f7fdc97fa9872446ca",
    "block_hash": "0000000000000000000c7d08493c6e23846c1af1719e334f7e9fcf599b78797d",
    "block_date": "2021-05-09",
    "inDegree": 3,
    "outDegree": 2
}
{
    "txid": "d81b4974b5501b0b8d5046104